## Bert topic single episode.

In [4]:
pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
#pip install  nltk

In [ ]:
#pip install transformers

In [ ]:
import pandas as pd
data=pd.read_csv("friends_transcripts.csv")
data.head()

,author,episode_number,episode_title,quote,quote_order,season
0,Monica,1,Monica Gets A Roommate,There's nothing to tell! He's just some guy I ...,0,1
1,Joey,1,Monica Gets A Roommate,"C'mon, you're going out with the guy! There's ...",1,1
2,Chandler,1,Monica Gets A Roommate,"All right Joey, be nice. So does he have a hum...",2,1
3,Phoebe,1,Monica Gets A Roommate,"Wait, does he eat chalk?",3,1
4,Phoebe,1,Monica Gets A Roommate,"Just, 'cause, I don't want her to go through w...",4,1


In [ ]:
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bertopic import BERTopic
from transformers import pipeline

# Download NLTK resources (only need to run once)
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Filter for Season 1, Episode 1
episode_1_data = data[(data["season"] == 1) & (data["episode_number"] == 1)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
episode_1_data.describe()

,episode_number,quote_order,season
count,300.0,300.000000,300.0
mean,1.0,150.646667,1.0
std,0.0,87.673775,0.0
min,1.0,0.000000,1.0
25%,1.0,74.750000,1.0
50%,1.0,150.500000,1.0
75%,1.0,226.250000,1.0
max,1.0,302.000000,1.0


In [ ]:
# Preprocess text data
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Tokenize
    words = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words]
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# Combine quotes into a single document and preprocess
texts = episode_1_data['quote'].apply(preprocess_text).tolist()


In [ ]:
texts

['there nothing tell he guy work',
 'cmon youre going guy there got ta something wrong',
 'right joey nice hump hump hairpiece',
 'wait eat chalk',
 'cause dont want go went carl oh',
 'okay everybody relax even date two people going dinner sex',
 'sound like date',
 'alright im back high school im standing middle cafeteria realize totally naked',
 'oh yeah dream',
 'look realize there phone',
 'instead',
 'thats right',
 'never dream',
 '',
 'sudden phone start ring dont know everybody start looking',
 'werent looking',
 'finally figure id better answer turn mother veryvery weird never call',
 'mortified hi',
 'guy say hello wan na kill',
 'okay sweetie',
 'feel like someone reached throat grabbed small intestine pulled mouth tied around neck',
 'cookie',
 'explaining others carol moved stuff today',
 'ohh',
 'ross let get coffee',
 'thanks',
 'ooh oh start pluck air front ross',
 'dont stop cleansing aura leave aura alone okay',
 'fine murky',
 'ill fine alright really everyone hope 

In [ ]:
# Define candidate labels
candidate_labels = [
    "Coffee", "Baby", "Child", "Death", "Cooking", "Turkey", "Thanksgiving", "Job", "Acting",
    "Dinosaur", "Paleontology", "Museum", "Sofa", "TV", "Jam", "Sex", "Breakup", "Romance",
    "Lesbian", "Divorce", "Marriage", "Friendship", "Coffeeshop", "Waiter", "Money", "Food",
    "Roommates", "Gossip", "New York", "Birthdays", "Hospital", "Birth", "Alcohol", "Cleaning"
]
# Initialize BERTopic and Zero-Shot Classifier
topic_model = BERTopic()
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [ ]:
topics, probabilities = topic_model.fit_transform(texts)  # Each quote as an individual document

In [ ]:
topic_words

{'Topic #0': ['okay',
  'na',
  'dont',
  'think',
  'go',
  'oh',
  'gon',
  'im',
  'yeah',
  'got'],
 'Topic #1': ['rachel',
  'monica',
  'chandler',
  'ross',
  'got',
  'la',
  'look',
  'joey',
  'monicas',
  'oh'],
 'Topic #2': ['im',
  'little',
  'know',
  'cant',
  'thing',
  'watch',
  'believe',
  'sorry',
  'god',
  'wooden'],
 'Topic #3': ['paul',
  'wine',
  'know',
  'guy',
  'hi',
  'everybody',
  'hey',
  'two',
  'tony',
  'snap']}

In [ ]:
# Step 2: Extract Topic Words
topic_words = {}
for topic in set(topics):
    if topic != -1:
        words = [word[0] for word in topic_model.get_topic(topic)]
        topic_words[f"Topic #{topic}"] = words


In [ ]:
# Step 3: Label each Topic for Episode 1 quotes
episode_1_labels = {}
for topic, words in topic_words.items():
    topic_text = " ".join(words)
    result = classifier(topic_text, candidate_labels)

    # Retrieve the top 4 labels and their confidence scores
    top_labels = result['labels'][:4]
    top_scores = result['scores'][:4]

    # Store the top 4 labels and scores in a list of tuples
    episode_1_labels[topic] = list(zip(top_labels, top_scores))

    # Print the results
    print(f"Topic {topic}:")
    for i in range(4):
        print(f"  Label {i+1}: {top_labels[i]} (Confidence: {top_scores[i]:.2f})")

# Optional: Convert to DataFrame for easier viewing
labeled_topics_df = pd.DataFrame([(topic, label, score) for topic, labels_scores in episode_1_labels.items() for label, score in labels_scores],
                                 columns=["Topic", "Label", "Confidence"])
print(labeled_topics_df)
# labeled_topics_df.to_csv("episode_1_labeled_topics.csv", index=False)


Topic Topic #0:
  Label 1: Child (Confidence: 0.07)
  Label 2: Museum (Confidence: 0.06)
  Label 3: Sofa (Confidence: 0.05)
  Label 4: Jam (Confidence: 0.05)
Topic Topic #1:
  Label 1: Lesbian (Confidence: 0.22)
  Label 2: Child (Confidence: 0.12)
  Label 3: TV (Confidence: 0.12)
  Label 4: Romance (Confidence: 0.09)
Topic Topic #2:
  Label 1: Child (Confidence: 0.64)
  Label 2: Gossip (Confidence: 0.03)
  Label 3: Breakup (Confidence: 0.03)
  Label 4: Sofa (Confidence: 0.03)
Topic Topic #3:
  Label 1: Alcohol (Confidence: 0.26)
  Label 2: Roommates (Confidence: 0.15)
  Label 3: Friendship (Confidence: 0.11)
  Label 4: Romance (Confidence: 0.05)
       Topic       Label  Confidence
0   Topic #0       Child    0.068509
1   Topic #0      Museum    0.056141
2   Topic #0        Sofa    0.048489
3   Topic #0         Jam    0.048027
4   Topic #1     Lesbian    0.219025
5   Topic #1       Child    0.122131
6   Topic #1          TV    0.116507
7   Topic #1     Romance    0.093874
8   Topic #2 

In [ ]:

# Optional: Save Episode 1 topics and labels
labeled_topics_df = pd.DataFrame([(topic, label, score) for topic, (label, score) in episode_1_labels.items()],
                                 columns=["Topic", "Label", "Confidence"])
print(labeled_topics_df)
# labeled_topics_df.to_csv("episode_1_labeled_topics.csv", index=False)

## BERT TOPIC SEASON 1

In [1]:
import pandas as pd
data=pd.read_csv("friends_transcripts.csv")
data.head()

,author,episode_number,episode_title,quote,quote_order,season
0,Monica,1.0,Monica Gets A Roommate,There's nothing to tell! He's just some guy I ...,0.0,1.0
1,Joey,1.0,Monica Gets A Roommate,"C'mon, you're going out with the guy! There's ...",1.0,1.0
2,Chandler,1.0,Monica Gets A Roommate,"All right Joey, be nice. So does he have a hum...",2.0,1.0
3,Phoebe,1.0,Monica Gets A Roommate,"Wait, does he eat chalk?",3.0,1.0
4,Phoebe,1.0,Monica Gets A Roommate,"Just, 'cause, I don't want her to go through w...",4.0,1.0


In [2]:
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bertopic import BERTopic
from transformers import pipeline

# Download NLTK resources (only need to run once)
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mahes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mahes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mahes\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\mahes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:

# Preprocess text data for each quote
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Tokenize
    words = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words]
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)


In [4]:
s10 = data[(data["season"] == 10)]
s9 = data[(data["season"] == 9)]
s8 = data[(data["season"] == 8)]
s7 = data[(data["season"] == 7)]
s6 = data[(data["season"] == 6)]
s5 = data[(data["season"] == 5)]
s4 = data[(data["season"] == 4)]
s3 = data[(data["season"] == 3)]
s2 = data[(data["season"] == 2)]
s1 = data[(data["season"] == 1)]

In [5]:
# Updated candidate labels for zero-shot classification
candidate_labels = [
    "Coffee","Child", "Death", "Cooking", "Thanksgiving", "Job", "Acting",
    "Dinosaur", "Paleontology", "Museum", "Sofa", "TV", "Sex", "Breakup", "Romance", "Divorce", "Marriage", "Friendship", "Waiter", "Money", "Food",
    "Roommates", "Gossip", "New York", "Birthdays", "Hospital", "Birth", "Alcohol", "Cleaning", "Clothes", "Fight"
]

# Initialize BERTopic and Zero-Shot Classifier
topic_model = BERTopic()
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device='cuda')

In [6]:
def topic_modeling(season_data, season_no):
    # Final DataFrame to store results
    results = []

    # Process each episode individually
    for episode in season_data['episode_number'].unique():
        episode_data = season_data[season_data['episode_number'] == episode]
        texts = episode_data['quote'].apply(preprocess_text).tolist()

        # Step 1: Fit BERTopic on the quotes from this episode
        topics, probabilities = topic_model.fit_transform(texts)

        # Step 2: Extract Topic Words for each unique topic in this episode
        topic_words = {}
        for topic in set(topics):
            if topic != -1:
                words = [word[0] for word in topic_model.get_topic(topic)]
                topic_words[topic] = words  # Store as int keys, not as strings like "Topic #0"

        # Step 3: Label each Topic for the current episode
        for topic, words in topic_words.items():
            topic_text = " ".join(words)
            result = classifier(topic_text, candidate_labels)

            # Retrieve the top 4 labels and their confidence scores
            top_labels = result['labels'][:4]
            top_scores = result['scores'][:4]

            # Store each label, score, and topic words for the topic in the final DataFrame
            for i in range(4):
                results.append({
                    "Season": season_no,  # Adding the season number
                    "Episode": episode,
                    "Topic": topic,
                    "Label": top_labels[i],
                    "Confidence": top_scores[i],
                    "Topic Words": words  # Directly add the words for the topic
                })
    return results

In [7]:
res_s10 = pd.DataFrame(topic_modeling(s10, 10))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [8]:
res_s9 = pd.DataFrame(topic_modeling(s9, 9))

In [9]:
res_s8 = pd.DataFrame(topic_modeling(s8, 8))

In [21]:
res_s10.head()

,Season,Episode,Topic,Label,Confidence,Topic Words
0,10,1.0,0,Job,0.228667,"[joey, wall, chandler, phoebe, know, rachel, w..."
1,10,1.0,0,Lesbian,0.144153,"[joey, wall, chandler, phoebe, know, rachel, w..."
2,10,1.0,0,Friendship,0.109556,"[joey, wall, chandler, phoebe, know, rachel, w..."
3,10,1.0,0,Romance,0.099205,"[joey, wall, chandler, phoebe, know, rachel, w..."
4,10,1.0,1,Child,0.058443,"[okay, dont, night, well, good, mean, alright,..."


In [10]:
res_s7 = pd.DataFrame(topic_modeling(s7, 7))
res_s6 = pd.DataFrame(topic_modeling(s6, 6))
res_s5 = pd.DataFrame(topic_modeling(s5, 5))
res_s4 = pd.DataFrame(topic_modeling(s4, 4))
res_s3 = pd.DataFrame(topic_modeling(s3, 3))
res_s2 = pd.DataFrame(topic_modeling(s2, 2))

In [11]:
res_s1 = pd.DataFrame(topic_modeling(data[data["season"] == 1], 1))

In [12]:
topic_modelling_results = pd.concat([res_s1, res_s2, res_s3, res_s4, res_s5, res_s6, res_s7, res_s8, res_s9, res_s10])

In [13]:
topic_modelling_results.to_csv("friends_topic_modeling_results.csv", index=False)

In [14]:
topic_modelling_results.sort_values(by=['Season', 'Episode', 'Confidence'], ascending=[True, True, False], inplace=True)

In [15]:
# Take only the top 4 labels for each episode
topic_modelling_results.groupby(["Season", "Episode"]).head(4).sort_values(by=['Season', 'Episode', 'Confidence'], ascending=[True, True, False])

,Season,Episode,Topic,Label,Confidence,Topic Words
8,1,1.0,2,Job,0.657898,"[go, think, job, dont, work, okay, right, get,..."
0,1,1.0,0,Coffee,0.482765,"[coffee, rachel, chandler, monica, joey, la, m..."
20,1,1.0,5,TV,0.438423,"[paul, wine, guy, everybody, hi, hey, know, tw..."
1,1,1.0,0,Food,0.215294,"[coffee, rachel, chandler, monica, joey, la, m..."
32,1,2.0,2,Child,0.362912,"[ross, rachel, people, parent, big, change, co..."
...,...,...,...,...,...,...
285,10,16.0,1,Acting,0.106447,"[yeah, oh, god, ok, well, damnit, flip, uhhmm,..."
292,10,17.0,0,Child,0.830605,"[baby, chandler, duck, chick, table, guy, phoe..."
332,10,17.0,10,Food,0.763704,"[okay, egg, roll, thing, youre, yeah, , , , ]"
312,10,17.0,5,Job,0.493793,"[joe, ow, rach, aw, boy, uh, , , , ]"


In [2]:
import pandas as pd

In [3]:
emotions = pd.read_csv("emotions_hopefully_final.csv")

In [13]:
topics = pd.read_csv("topic_modelling_hopefully_final.csv")

In [5]:
topics

,Season,Episode,Topic,Label,Confidence,Topic Words
0,1,1.0,0,Coffee,0.482765,"['coffee', 'rachel', 'chandler', 'monica', 'jo..."
1,1,1.0,0,Food,0.215294,"['coffee', 'rachel', 'chandler', 'monica', 'jo..."
2,1,1.0,0,Child,0.112966,"['coffee', 'rachel', 'chandler', 'monica', 'jo..."
3,1,1.0,0,Roommates,0.066807,"['coffee', 'rachel', 'chandler', 'monica', 'jo..."
4,1,1.0,1,Museum,0.069503,"['okay', 'yeah', 'oh', 'sure', 'know', 'yes', ..."
...,...,...,...,...,...,...
4411,10,17.0,12,Gossip,0.066765,"['ross', 'going', 'might', 'feel', 'sorry', 'r..."
4412,10,17.0,13,Gossip,0.302515,"['rachel', 'talk', 'green', 'bicycle', 'deathc..."
4413,10,17.0,13,Death,0.157050,"['rachel', 'talk', 'green', 'bicycle', 'deathc..."
4414,10,17.0,13,Acting,0.075377,"['rachel', 'talk', 'green', 'bicycle', 'deathc..."


In [6]:
emotions.columns

Index(['author', 'episode_number', 'episode_title', 'quote', 'quote_order',
       'season', 'emotion1', 'emotion1_prob', 'emotion2', 'emotion2_prob',
       'final_emotion', 'final_emotion_new'],
      dtype='object')

In [14]:
topics.columns = ['season', 'episode_number', 'topic', 'label', 'confidence', 'topic_words']

In [15]:
topics

,season,episode_number,topic,label,confidence,topic_words
0,1,1,0,Coffee,0.482765,"['coffee', 'rachel', 'chandler', 'monica', 'jo..."
1,1,1,0,Food,0.215294,"['coffee', 'rachel', 'chandler', 'monica', 'jo..."
2,1,1,0,Child,0.112966,"['coffee', 'rachel', 'chandler', 'monica', 'jo..."
3,1,1,0,Roommates,0.066807,"['coffee', 'rachel', 'chandler', 'monica', 'jo..."
4,1,1,1,Museum,0.069503,"['okay', 'yeah', 'oh', 'sure', 'know', 'yes', ..."
...,...,...,...,...,...,...
4411,10,17,12,Gossip,0.066765,"['ross', 'going', 'might', 'feel', 'sorry', 'r..."
4412,10,17,13,Gossip,0.302515,"['rachel', 'talk', 'green', 'bicycle', 'deathc..."
4413,10,17,13,Death,0.157050,"['rachel', 'talk', 'green', 'bicycle', 'deathc..."
4414,10,17,13,Acting,0.075377,"['rachel', 'talk', 'green', 'bicycle', 'deathc..."


In [16]:
# From each topic in 'topics', get the label with the highest confidence, for each season and episode
final_labels = topics.groupby(['season', 'episode_number', 'topic'])['confidence'].transform(max) == topics['confidence']


C:\Users\mahes\AppData\Local\Temp\ipykernel_18148\3368875295.py:2: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  final_labels = topics.groupby(['season', 'episode_number', 'topic'])['confidence'].transform(max) == topics['confidence']


In [18]:
topics['label_flag'] = final_labels

In [19]:
test = topics[topics['label_flag']==True]

In [20]:
test

,season,episode_number,topic,label,confidence,topic_words,label_flag
0,1,1,0,Coffee,0.482765,"['coffee', 'rachel', 'chandler', 'monica', 'jo...",True
4,1,1,1,Museum,0.069503,"['okay', 'yeah', 'oh', 'sure', 'know', 'yes', ...",True
8,1,1,2,Job,0.657898,"['go', 'think', 'job', 'dont', 'work', 'okay',...",True
12,1,1,3,Gossip,0.124361,"['cut', 'cant', 'thing', 'believe', 'watch', '...",True
16,1,1,4,Child,0.205825,"['na', 'wan', 'date', 'shoe', 'woman', 'phone'...",True
...,...,...,...,...,...,...,...
4396,10,17,9,Gossip,0.163927,"['god', 'oh', 'step', 'shhh', 'name', 'shoot',...",True
4400,10,17,10,Food,0.763704,"['okay', 'egg', 'roll', 'thing', 'youre', 'yea...",True
4404,10,17,11,Child,0.075251,"['yeah', 'yes', 'havent', 'clear', 'board', 'u...",True
4408,10,17,12,Breakup,0.149658,"['ross', 'going', 'might', 'feel', 'sorry', 'r...",True


In [8]:
# Take the top 4 labels for each episode and store them as a list corresponding to each episode

topics_top4 = topics[['season', 'episode_number', 'label', 'confidence']].groupby(["season", "episode_number"]).head(4).sort_values(by=['season', 'episode_number', 'confidence'], ascending=[True, True, False])

In [46]:
topics_top4

,season,episode_number,label,confidence
0,1,1.0,Coffee,0.482765
1,1,1.0,Food,0.215294
2,1,1.0,Child,0.112966
3,1,1.0,Roommates,0.066807
24,1,2.0,Breakup,0.075539
...,...,...,...,...
4351,10,16.0,Death,0.055112
4360,10,17.0,Child,0.830605
4361,10,17.0,Food,0.022411
4362,10,17.0,Birth,0.021872


In [32]:
frontend_data = pd.DataFrame()

In [33]:
frontend_data = test.groupby(["season", "episode_number"])['label'].apply(list).reset_index()

In [34]:
frontend_data['confidence'] = test.groupby(["season", "episode_number"])['confidence'].apply(list).reset_index()['confidence']

In [35]:
frontend_data

,season,episode_number,label,confidence
0,1,1,"[Coffee, Museum, Job, Gossip, Child, TV]","[0.482765168, 0.069502823, 0.65789783, 0.12436..."
1,1,2,"[Breakup, Child, Child, Gossip, Food]","[0.075539395, 0.100336947, 0.36291182, 0.22275..."
2,1,3,"[Gossip, Child, Child, Child, Friendship, Goss...","[0.3679353, 0.362183362, 0.076231778, 0.128311..."
3,1,4,"[Romance, Gossip]","[0.071423687, 0.097724177]"
4,1,5,"[Friendship, Sex]","[0.096595235, 0.235470906]"
...,...,...,...,...
221,10,13,"[Child, Birth]","[0.087734371, 0.330314368]"
222,10,14,"[Child, Cleaning, Friendship, Job, Roommates, ...","[0.418976575, 0.375140995, 0.502599597, 0.6037..."
223,10,15,"[Friendship, Roommates, Job]","[0.112285048, 0.200507477, 0.404895127]"
224,10,16,"[Breakup, TV, Friendship]","[0.115114152, 0.116761856, 0.16955933]"


In [38]:
frontend = pd.read_csv("frontend.csv")

In [39]:
frontend_data[['episode_title', 'emotion1', 'emotion1_prob', 'emotion2', 'emotion2_prob', 'summary']] = frontend[['episode_title', 'emotion1', 'emotion1_prob', 'emotion2', 'emotion2_prob', 'summary']]

In [40]:
frontend_data.to_csv('frontend_data.csv', index=False)

In [12]:
grouped = emotions.groupby(['season', 'episode_number'])

In [37]:
def get_top_emotions(group):
    emotions = pd.concat([
        group[['emotion1', 'emotion1_prob']].rename(columns={'emotion1': 'emotion', 'emotion2_prob': 'prob'}),
        group[['emotion2', 'emotion2_prob']].rename(columns={'emotion2': 'emotion', 'emotion2_prob': 'prob'})
    ])
    emotions_summary = emotions.groupby('emotion')['prob'].mean()
    
    # Get the top 2 emotions by mean probability
    top_2_emotions = emotions_summary.nlargest(2)
    
    # Return a dictionary with the top 2 emotions and their probabilities
    return pd.Series({
        'emotion1': top_2_emotions.index[0],
        'emotion1_prob': top_2_emotions.iloc[0],
        'emotion2': top_2_emotions.index[1],
        'emotion2_prob': top_2_emotions.iloc[1]
    })

In [38]:
episode_emotions = grouped.apply(get_top_emotions).reset_index()

C:\Users\mahes\AppData\Local\Temp\ipykernel_5272\218595579.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  episode_emotions = grouped.apply(get_top_emotions).reset_index()


In [103]:
episode_emotions

,season,episode_number,top_emotion_1,prob_1,top_emotion_2,prob_2
0,1.0,1.0,satisfied,0.323980,relaxed,0.292476
1,1.0,2.0,confused,0.287636,relaxed,0.286402
2,1.0,3.0,satisfied,0.283805,confused,0.277726
3,1.0,4.0,afraid,0.292932,satisfied,0.289345
4,1.0,5.0,relaxed,0.326254,confused,0.306292
...,...,...,...,...,...,...
197,10.0,13.0,afraid,0.315193,satisfied,0.281731
198,10.0,14.0,depressed,0.402260,confused,0.279827
199,10.0,15.0,satisfied,0.320741,afraid,0.308480
200,10.0,16.0,afraid,0.313526,satisfied,0.292830


In [39]:
emotions.groupby(['season','episode_number']).head(1)['episode_title'].reset_index(drop=True)

0                 Monica Gets A Roommate
1                The Sonogram At The End
2                              The Thumb
3                  George Stephanopoulos
4      The East German Laundry Detergent
                     ...                
221                   Joey Speaks French
222                    Princess Consuela
223                         Estelle Dies
224            Rachel's Going Away Party
225            The Last One, Part I & II
Name: episode_title, Length: 226, dtype: object

In [41]:
pd.concat([frontend_data, emotions.groupby(['season','episode_number']).head(1)['episode_title'].reset_index(drop=True) ,episode_emotions[['emotion1', 'emotion1_prob', 'emotion2', 'emotion2_prob']]], axis=1).to_csv('frontend.csv', index=False)

In [117]:
frontend = pd.concat([frontend_data, emotions.groupby(['season','episode_number']).head(1)['episode_title'].reset_index(drop=True) ,episode_emotions[['top_emotion_1', 'prob_1', 'top_emotion_2', 'prob_2']]], axis=1)

In [1]:
import pandas as pd

In [42]:
frontend = pd.read_csv('frontend.csv')

In [18]:
frontend['label'][0]

"['Coffee', 'Food', 'Child', 'Roommates']"

In [19]:
import ast

In [20]:
frontend['label'].apply(ast.literal_eval)

0            [Coffee, Food, Child, Roommates]
1               [Breakup, Child, Sofa, Fight]
2        [Gossip, Friendship, Romance, Fight]
3        [Romance, Gossip, Roommates, Acting]
4      [Friendship, Child, Museum, Roommates]
                        ...                  
221         [Child, Museum, Sofa, Friendship]
222           [Child, Roommates, Gossip, Job]
223      [Friendship, Romance, Child, Museum]
224           [Breakup, Gossip, Child, Death]
225             [Child, Food, Birth, Romance]
Name: label, Length: 226, dtype: object

In [43]:
frontend

,season,episode_number,label,confidence,episode_title,emotion1,emotion1_prob,emotion2,emotion2_prob
0,1,1.0,"['Coffee', 'Food', 'Child', 'Roommates']","[0.4827651679515838, 0.2152937948703766, 0.112...",Monica Gets A Roommate,surprise,0.252246,joy,0.184626
1,1,2.0,"['Breakup', 'Child', 'Sofa', 'Fight']","[0.0755393952131271, 0.075136661529541, 0.0589...",The Sonogram At The End,surprise,0.250213,love,0.195201
2,1,3.0,"['Gossip', 'Friendship', 'Romance', 'Fight']","[0.367935299873352, 0.1213470697402954, 0.0718...",The Thumb,surprise,0.238586,sadness,0.199354
3,1,4.0,"['Romance', 'Gossip', 'Roommates', 'Acting']","[0.0714236870408058, 0.0690843090415, 0.068384...",George Stephanopoulos,surprise,0.250511,love,0.191629
4,1,5.0,"['Friendship', 'Child', 'Museum', 'Roommates']","[0.0965952351689338, 0.0807569995522499, 0.059...",The East German Laundry Detergent,surprise,0.246075,annoyed,0.185149
...,...,...,...,...,...,...,...,...,...
221,10,13.0,"['Child', 'Museum', 'Sofa', 'Friendship']","[0.0877343714237213, 0.0618200339376926, 0.056...",Joey Speaks French,surprise,0.258221,love,0.184354
222,10,14.0,"['Child', 'Roommates', 'Gossip', 'Job']","[0.4189765751361847, 0.2942845225334167, 0.040...",Princess Consuela,surprise,0.256565,love,0.214950
223,10,15.0,"['Friendship', 'Romance', 'Child', 'Museum']","[0.1122850477695465, 0.0906340628862381, 0.074...",Estelle Dies,sadness,0.232092,surprise,0.218284
224,10,16.0,"['Breakup', 'Gossip', 'Child', 'Death']","[0.115114152431488, 0.0611238926649093, 0.0571...",Rachel's Going Away Party,surprise,0.262877,sadness,0.205980


In [45]:
summaries = pd.read_csv('summaries.csv', encoding='latin1')

In [47]:
frontend['summary'] = summaries['summary']

In [48]:
frontend.to_csv('frontend.csv')

In [49]:
emotions[(emotions['season']==2) & (emotions['episode_number']==19)]

,author,episode_number,episode_title,quote,quote_order,season,emotion1,emotion1_prob,emotion2,emotion2_prob
10074,CHANDLER,19.0,Eddie Won't Go,Hey Eddie. Daahh!! What're you doin' here?,0.0,2.0,annoyed,0.491231,surprise,0.349102
10075,EDDIE,19.0,Eddie Won't Go,"Nothin' roomie, just watchin' you sleep.",1.0,2.0,surprise,0.504827,disgust,0.116587
10076,CHANDLER,19.0,Eddie Won't Go,Why?,2.0,2.0,surprise,0.645612,annoyed,0.129567
10077,EDDIE,19.0,Eddie Won't Go,"Makes me feel um, peaceful, heh-heh, please.",3.0,2.0,joy,0.480936,surprise,0.210207
10078,CHANDLER,19.0,Eddie Won't Go,I can't sleep now.,4.0,2.0,annoyed,0.622624,surprise,0.190956
...,...,...,...,...,...,...,...,...,...,...
10334,CHANDLER,19.0,Eddie Won't Go,"Hey look, are we gonna have to bring this out ...",260.0,2.0,annoyed,0.824889,surprise,0.106817
10335,JOEY,19.0,Eddie Won't Go,He paid a lot of money for it.,261.0,2.0,surprise,0.482645,joy,0.133402
10336,CHANDLER,19.0,Eddie Won't Go,I'm gonna hold him a different way. Look I don...,262.0,2.0,annoyed,0.599537,surprise,0.205217
10337,JOEY,19.0,Eddie Won't Go,"Well, I had a whole ceramic zoo thing goin' ov...",263.0,2.0,annoyed,0.656438,disgust,0.210002
